Outras similaridades: Cos, Ovl, Jacc, Dice

Semantic Similarity

These models find semantically similar sentences within one language or across languages:

**distiluse-base-multilingual-cased-v2**: Multilingual knowledge distilled version of multilingual Universal Sentence Encoder. While the original mUSE model only supports 16 languages, this multilingual knowledge distilled version supports 50+ languages.

**xlm-r-distilroberta-base-paraphrase-v1** - Multilingual version of distilroberta-base-paraphrase-v1, trained on parallel data for 50+ languages.

**xlm-r-bert-base-nli-stsb-mean-tokens**: Produces similar embeddings as the bert-base-nli-stsb-mean-token model. Trained on parallel data for 50+ languages.

**distilbert-multilingual-nli-stsb-quora-ranking** - Multilingual version of distilbert-base-nli-stsb-quora-ranking. Fine-tuned with parallel data for 50+ languages.

**T-Systems-onsite/cross-en-de-roberta-sentence-transformer** - Multilingual model for English an German. [More]

# Imports e métodos necessários

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [10]:
import pandas as pd 
import xml.etree.ElementTree as et 

from scipy.stats import pearsonr

def parse_xml(xml_file):
    """Parse xml to pandas dataframe."""
    xtree = et.parse(xml_file)
    xroot = xtree.getroot() 

    df_cols = ['id', 't', 'h', 'similarity']
    rows = []

    for node in xroot:
        id_ = node.attrib.get("id")
        similarity = node.attrib.get("similarity")
        t = node.find("t").text
        h = node.find("h").text

        rows.append({
            "id": id_,
            "t": t, 
            "h": h,
            "similarity": similarity
        })
    return pd.DataFrame(rows, columns=df_cols, dtype=float)

def eval_similarity(pairs_gold, pairs_sys):
    '''
    Evaluate the semantic similarity output of the system against a gold score. 
    Results are printed to stdout.
    '''
    
    gold_values = np.array(pairs_gold)
    sys_values = np.array(pairs_sys)
    pearson = pearsonr(gold_values, sys_values)[0]
    absolute_diff = gold_values - sys_values
    mse = (absolute_diff ** 2).mean()
    
    print()
    print('Similarity evaluation')
    print('Pearson\t\tMean Squared Error')
    print('-------\t\t------------------')
    print('{:7.3f}\t\t{:18.2f}'.format(pearson, mse))

# Carregando os dados

In [11]:
!ls ../data/assin

assin-ptbr-dev.xml   assin-ptbr-train.xml  assin-ptpt-test.xml
assin-ptbr-test.xml  assin-ptpt-dev.xml    assin-ptpt-train.xml


In [12]:
!ls ../data/assin2

assin2-blind-test.xml  assin2-dev.xml  assin2-test.xml	assin2-train-only.xml


In [13]:
df_ptbr_train = parse_xml('../data/assin/assin-ptbr-train.xml')
df_ptbr_dev = parse_xml('../data/assin/assin-ptbr-dev.xml')
df_ptbr_test = parse_xml('../data/assin/assin-ptbr-test.xml')

df_ptpt_train = parse_xml('../data/assin/assin-ptpt-train.xml')
df_ptpt_dev = parse_xml('../data/assin/assin-ptpt-dev.xml')
df_ptpt_test = parse_xml('../data/assin/assin-ptpt-test.xml')

df_ptbr2_train = parse_xml('../data/assin2/assin2-train-only.xml')
df_ptbr2_dev = parse_xml('../data/assin2/assin2-dev.xml')
df_ptbr2_test = parse_xml('../data/assin2/assin2-test.xml')

In [14]:
df_ptbr_train = pd.concat([df_ptbr_train, df_ptpt_train, df_ptbr2_train])
df_ptbr_dev = pd.concat([df_ptbr_dev, df_ptpt_dev, df_ptbr2_dev])
# df_ptbr_train = pd.concat([df_ptbr_train, df_ptpt_train, df_ptbr2_train])

In [15]:
print(f'assin-ptbr-train: {df_ptbr_train.shape}')
print(f'assin-ptbr-dev: {df_ptbr_dev.shape}')
print()
print(f'assin-ptbr-test: {df_ptbr_test.shape}')
print(f'assin-ptpt-test: {df_ptpt_test.shape}')
print(f'assin-ptbr2-test: {df_ptbr2_test.shape}')

assin-ptbr-train: (11500, 4)
assin-ptbr-dev: (1500, 4)

assin-ptbr-test: (2000, 4)
assin-ptpt-test: (2000, 4)
assin-ptbr2-test: (2448, 4)


In [8]:
df_ptbr_train.head()

,id,t,h,similarity
0,1.0,"A gente faz o aporte financeiro, é como se a e...",Fernando Moraes afirma que não tem vínculo com...,2.00
1,2.0,"Em 2013, a história de como Walt Disney conven...",P.L.Travers era completamente contra a adaptaç...,2.25
2,3.0,"David Silva bateu escanteio, Kompany escalou a...","David Silva cobrou escanteio, o zagueiro se ap...",3.75
3,4.0,"Para os ambientalistas, as metas anunciadas pe...","Dilma aproveitou seu discurso ontem, na Confer...",2.75
4,5.0,"De acordo com a PM, por volta das 10h30 havia ...",O protesto encerrou por volta de 12h15 (horári...,2.00


# Testes

## Fine-tuning Sentence-BERT

https://www.sbert.net/docs/training/overview.html

https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/sts/training_stsbenchmark_continue_training.py

### xlm-r-distilroberta-base-paraphrase-v1

In [18]:
from sklearn.metrics.pairwise import  cosine_similarity
from scipy.stats import spearmanr

def test_evaluation(df_ptbr_test):
    t_embeddings = model.encode(df_ptbr_test['t'])#,show_progress_bar=True)
    h_embeddings = model.encode(df_ptbr_test['h'])#,show_progress_bar=True)

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    eval_similarity(pairs_gold, pairs_sys)
    
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

In [17]:
#df_ptbr_train = pd.concat([df_ptbr_train, df_ptbr_dev])
df_ptbr_train.shape, df_ptbr_dev.shape

((11500, 4), (1500, 4))

In [19]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer



# Read the dataset
for i in range(1, 10+1):
    train_batch_size = 16
    num_epochs = 4 # tentar distilroberta large
    model_save_path = f'../data/sbert_finetuning_assin2_xlm-r-distilroberta-base-paraphrase-v1-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)


    model = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1', device='cpu')

    train_samples = []
    dev_samples = []
    #test_samples = []

    for i, row in df_ptbr_train.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        train_samples.append(inp_example)

    for i, row in df_ptbr_dev.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        dev_samples.append(inp_example)

    #for i, row in df_ptbr_test.iterrows():
    #    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    #    test_samples.append(inp_example)



    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)
    
    print('####################################################################################')
    print(f'Test {i}: ASSIN pt-br:')
    test_evaluation(df_ptbr_test)
    
    print(f'Test {i}: ASSIN pt-pt:')
    test_evaluation(df_ptpt_test)
    
    print(f'Test {i}: ASSIN 2:')
    test_evaluation(df_ptbr2_test)
    print('####################################################################################')
    



####################################################################################
Test 499: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.792		              0.30
Spearman correlation:   0.779
Test 499: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.810		              0.49
Spearman correlation:   0.802
Test 499: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.853		              0.41
Spearman correlation:   0.817
####################################################################################




####################################################################################
Test 499: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.793		              0.29
Spearman correlation:   0.780
Test 499: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.809		              0.50
Spearman correlation:   0.800
Test 499: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.850		              0.41
Spearman correlation:   0.814
####################################################################################




####################################################################################
Test 499: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.792		              0.30
Spearman correlation:   0.779
Test 499: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.808		              0.49
Spearman correlation:   0.799
Test 499: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.851		              0.41
Spearman correlation:   0.815
####################################################################################




####################################################################################
Test 499: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.795		              0.29
Spearman correlation:   0.781
Test 499: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.811		              0.49
Spearman correlation:   0.801
Test 499: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.852		              0.41
Spearman correlation:   0.818
####################################################################################




####################################################################################
Test 499: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.792		              0.30
Spearman correlation:   0.778
Test 499: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.809		              0.50
Spearman correlation:   0.800
Test 499: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.852		              0.41
Spearman correlation:   0.818
####################################################################################




####################################################################################
Test 499: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.795		              0.29
Spearman correlation:   0.781
Test 499: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.809		              0.49
Spearman correlation:   0.800
Test 499: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.852		              0.41
Spearman correlation:   0.816
####################################################################################




####################################################################################
Test 499: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.793		              0.29
Spearman correlation:   0.779
Test 499: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.811		              0.49
Spearman correlation:   0.801
Test 499: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.852		              0.41
Spearman correlation:   0.816
####################################################################################




####################################################################################
Test 499: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.790		              0.30
Spearman correlation:   0.776
Test 499: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.809		              0.49
Spearman correlation:   0.801
Test 499: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.853		              0.41
Spearman correlation:   0.817
####################################################################################




####################################################################################
Test 499: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.793		              0.30
Spearman correlation:   0.779
Test 499: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.811		              0.49
Spearman correlation:   0.802
Test 499: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.853		              0.41
Spearman correlation:   0.819
####################################################################################




####################################################################################
Test 499: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.793		              0.29
Spearman correlation:   0.781
Test 499: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.808		              0.49
Spearman correlation:   0.799
Test 499: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.851		              0.41
Spearman correlation:   0.817
####################################################################################


In [20]:
df_ptbr_train = pd.concat([df_ptbr_train, df_ptbr_dev])
df_ptbr_train.shape, df_ptbr_dev.shape

((13000, 4), (1500, 4))

In [21]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer

train_samples = []
    #dev_samples = []
    #test_samples = []

for i, row in df_ptbr_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)

# Read the dataset
for i in range(1, 10+1):
    train_batch_size = 16
    num_epochs = 4 # tentar distilroberta large
    model_save_path = f'../data/sbert_finetuning_assin2_xlm-r-distilroberta-base-paraphrase-v1-all-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)


    model = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1', device='cpu')

    #train_samples = []
    #dev_samples = []
    #test_samples = []

    #for i, row in df_ptbr_train.iterrows():
    #    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    #    train_samples.append(inp_example)

    #for i, row in df_ptbr_dev.iterrows():
    #    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    #    dev_samples.append(inp_example)

    #for i, row in df_ptbr_test.iterrows():
    #    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    #    test_samples.append(inp_example)



    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    #evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              #evaluator=evaluator,
              epochs=num_epochs,
              #evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)
    
    print('####################################################################################')
    print(f'Test {i}: ASSIN pt-br:')
    test_evaluation(df_ptbr_test)
    
    print(f'Test {i}: ASSIN pt-pt:')
    test_evaluation(df_ptpt_test)
    
    print(f'Test {i}: ASSIN 2:')
    test_evaluation(df_ptbr2_test)
    print('####################################################################################')



####################################################################################
Test 1: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.798		              0.29
Spearman correlation:   0.784
Test 1: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.810		              0.49
Spearman correlation:   0.802
Test 1: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.851		              0.41
Spearman correlation:   0.816
####################################################################################




####################################################################################
Test 2: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.796		              0.29
Spearman correlation:   0.782
Test 2: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.810		              0.49
Spearman correlation:   0.801
Test 2: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.850		              0.42
Spearman correlation:   0.816
####################################################################################




####################################################################################
Test 3: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.796		              0.29
Spearman correlation:   0.783
Test 3: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.811		              0.48
Spearman correlation:   0.802
Test 3: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.848		              0.42
Spearman correlation:   0.814
####################################################################################




####################################################################################
Test 4: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.797		              0.29
Spearman correlation:   0.784
Test 4: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.811		              0.49
Spearman correlation:   0.803
Test 4: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.851		              0.41
Spearman correlation:   0.815
####################################################################################




####################################################################################
Test 5: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.796		              0.29
Spearman correlation:   0.784
Test 5: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.811		              0.49
Spearman correlation:   0.802
Test 5: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.849		              0.41
Spearman correlation:   0.814
####################################################################################




####################################################################################
Test 6: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.798		              0.29
Spearman correlation:   0.785
Test 6: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.812		              0.48
Spearman correlation:   0.804
Test 6: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.850		              0.42
Spearman correlation:   0.816
####################################################################################




####################################################################################
Test 7: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.797		              0.29
Spearman correlation:   0.785
Test 7: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.809		              0.49
Spearman correlation:   0.801
Test 7: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.850		              0.42
Spearman correlation:   0.814
####################################################################################




####################################################################################
Test 8: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.798		              0.29
Spearman correlation:   0.784
Test 8: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.813		              0.48
Spearman correlation:   0.804
Test 8: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.851		              0.41
Spearman correlation:   0.817
####################################################################################




####################################################################################
Test 9: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.797		              0.29
Spearman correlation:   0.783
Test 9: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.811		              0.49
Spearman correlation:   0.803
Test 9: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.852		              0.41
Spearman correlation:   0.818
####################################################################################




####################################################################################
Test 10: ASSIN pt-br:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.796		              0.29
Spearman correlation:   0.784
Test 10: ASSIN pt-pt:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.813		              0.48
Spearman correlation:   0.803
Test 10: ASSIN 2:

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.851		              0.41
Spearman correlation:   0.819
####################################################################################
